# Análisis de candidatos
En este notebook se analiza el dataset correspondiente a Candidates.csv

### Tareas vigentes
[ ] Sacar distribución de salarios y agruparlos por clases. 
[ ] Analizar qué contiene específicamente Studies y Experiences. 
[ ] Replantear las clases de City, generar clase de ciudades extranjeras. Es necesario correlacionar con las ciudades de las vacantes (opcional).
[ ] Puede ser buena idea usar la fecha de nacimiento, en Colombia aún se fijan mucho en la edad.


In [1]:
# Importar herramientas
import hackaUtils

In [2]:
# Importar dataset
dt_name = 'Candidates'
dataset = hackaUtils.get_dataset([dt_name])[dt_name]

## Análisis de información faltante
### Cantidad de registros vacíos por campo
Se puede observar que en el 94% de los registros, hace falta title_or_profession, en el 92% de los casos falta available_to_move, en el 84% de los casos falta civil_status y en el 71% de los casos falta profile_description. Son los datos más relevantes, es necesario analizar si se tienen en cuenta o no estas variables.

In [3]:
# Se cuenta cuantos NaN o Null hay por campo en el dataset
hackaUtils.count_empty_records(dataset)

		Missing data per field
Total records		 548363
Missing data		% of total	Field name(23)
517270			94%		title_or_profession
503146			92%		available_to_move
462981			84%		civil_status
462709			84%		has_video
386665			71%		profile_description
190738			35%		salary
81329			15%		birthdate
75033			14%		gender
51788			9%		identification_type
46607			8%		identification_number
26230			5%		city
21523			4%		country_birth
16323			3%		education_level
8096			1%		phone
31			0%		last_name
9			0%		first_name
0			0%		psy_tests
0			0%		experiences
0			0%		studies
0			0%		without_studies
0			0%		without_experience
0			0%		email


## Registros que tienen listas vacías por campo

Se puede observar que el 89% de los registros no tiene psy_tests, el 40% no tiene *experiences* y el 38% no tiene *studies*

In [4]:
# Se cuenta cuantos [] hay por campo en el dataset
hackaUtils.count_empty_lists(dataset)

Empty lists
Total records		 548363
Missing data		% of total	Field name(23)
0			0%		id
0			0%		email
0			0%		first_name
0			0%		last_name
0			0%		phone
0			0%		birthdate
0			0%		gender
0			0%		identification_type
0			0%		identification_number
0			0%		country_birth
0			0%		city
0			0%		education_level
0			0%		salary
0			0%		profile_description
0			0%		without_experience
0			0%		without_studies
0			0%		title_or_profession
0			0%		available_to_move
0			0%		civil_status
0			0%		has_video
206536			38%		studies
220280			40%		experiences
485719			89%		psy_tests


#### Eliminar variables con mucha ausencia o poca relevancia
Se procede a eliminar en primera instancia las variables: *title_or_profession*, *available_to_move*, *civil_status*, *has_video*, *profile_description*, *birthdate*, *identification_type*, *identification_number* y por contener un 89% de registros conformados por listas vacías, se elimina *psy_tests*. La vairable *gender* se deja hasta que se compruebe la correlación entre genero y cargo. Por poca relevancia se descartan las variables *email*, *first_name*, *last_name*, *phone*.

De 23 campos, pasamos a 10.

In [5]:
columns = ['title_or_profession','available_to_move','civil_status','has_video','profile_description','birthdate','identification_type','identification_number','psy_tests','email','first_name','last_name','phone']
filtered_dt = hackaUtils.drop_columns(dataset, columns)

## Análisis de variables habilitantes
Una variables habilitante es aquella que es fundamental para la existencia de un registro. Es decir, si una variables X es fundamental para el registro Y, si X tiene un valor incoherente o inexistente, se permite descartar al registro Y.
Esto lo podemos usar para reducir el tamaño del dataset y refinar los datos de entrada.

## Análisis individual de las variables


### 1. Gender

In [6]:
# Gender
column = 'gender'
props = hackaUtils.count_record_per_class(filtered_dt, column)



Field values (%):  gender
female          0.498690
male            0.363985
NaN             0.136831
unknown         0.000286
other_gender    0.000208
Name: gender, dtype: float64

Field values (value):  gender
female          273463
male            199596
NaN              75033
unknown            157
other_gender       114
Name: gender, dtype: int64



__Conclusión:__ Se puede observar que casi el 50% de los candidatos son mujeres. Entre hombres y mujeres hay aproximadamente un 86% de los valores del campo gender. Se propone agrupar los valores que no sean 'male' y 'female' en la categoría 'others'. En este caso, ya que *unknown* y *other_gender* tienen valores muy pequeños, no vale la pena separarlos de NaN.


In [8]:
# Reemplazar NaN, unknown y other_gender por others
classes = hackaUtils.get_minor_classes(props,2)
group_class = 'others'
hackaUtils.replace_minority_values(dt = filtered_dt, field = column, classes = classes, new_class = group_class)
# Check distribution again
props = hackaUtils.count_record_per_class(filtered_dt, column)




Field values (%):  gender
female    0.498690
male      0.363985
others    0.137325
Name: gender, dtype: float64

Field values (value):  gender
female    273463
male      199596
others     75304
Name: gender, dtype: int64


### 2. Country birth

In [9]:
# Country birth
column = 'country_birth'
props = hackaUtils.count_record_per_class(filtered_dt, column)



Field values (%):  country_birth
Colombia              0.904264
NaN                   0.039250
Venezuela             0.037685
Dominican Republic    0.009955
Peru                  0.003735
                        ...   
Vanuatu               0.000002
Myanmar               0.000002
Lithuania             0.000002
Bhutan                0.000002
India                 0.000002
Name: country_birth, Length: 99, dtype: float64

Field values (value):  country_birth
Colombia              495865
NaN                    21523
Venezuela              20665
Dominican Republic      5459
Peru                    2048
                       ...  
Vanuatu                    1
Myanmar                    1
Lithuania                  1
Bhutan                     1
India                      1
Name: country_birth, Length: 99, dtype: int64



__Conclusión:__ Debido a que más del 90% de los registros indican que los candidatos son de Colombia y el segundo valor más preponderante es NaN, se eliminará el campo de *country_birth*

In [10]:
# Eliminando el campo country_birth
columns = ['country_birth']
filtered_dt = hackaUtils.drop_columns(filtered_dt, columns)

### 3. City

In [11]:
# City
column = 'city'
props = hackaUtils.count_record_per_class(filtered_dt, column)



Field values (%):  city
Bogota             0.257891
Medellin           0.118828
Cali               0.056360
Barranquilla       0.049019
NaN                0.047833
                     ...   
Laval              0.000002
Tinaquillo         0.000002
Seattle            0.000002
Manchester         0.000002
Roquetas de Mar    0.000002
Name: city, Length: 1434, dtype: float64

Field values (value):  city
Bogota             141418
Medellin            65161
Cali                30906
Barranquilla        26880
NaN                 26230
                    ...  
Laval                   1
Tinaquillo              1
Seattle                 1
Manchester              1
Roquetas de Mar         1
Name: city, Length: 1434, dtype: int64



__Conclusión:__ Se aproximan todas las ciudades, a partir de los registros con NaN, a una sola clase para facilitar las cosas. Cabe resaltar que es buena idea hacer un análisis cruzado de las ciudades de los candidatos, con las ciudades de las vacantes. Ya que la clase 'others' queda con más del 50% de registros, hay que analizar si es buena idea incluir más o volver NaN una clase formal, reemplazando los valores de NaN por algo como 'undefined'.


In [12]:
# Reemplazar NaN y otras ciudades minoritarias por others
classes = hackaUtils.get_minor_classes(props,4)
group_class = 'others'
hackaUtils.replace_minority_values(dt = filtered_dt, field = column, classes = classes, new_class = group_class)
# Check distribution again
props = hackaUtils.count_record_per_class(filtered_dt, column)



Field values (%):  city
others          0.517901
Bogota          0.257891
Medellin        0.118828
Cali            0.056360
Barranquilla    0.049019
Name: city, dtype: float64

Field values (value):  city
others          283998
Bogota          141418
Medellin         65161
Cali             30906
Barranquilla     26880
Name: city, dtype: int64


### 4. Education level

In [9]:
# Education level
column = 'education_level'
props = hackaUtils.count_record_per_class(filtered_dt, column)
# Cambiar listas vacías por clase 'undefined'
nan_class = 'undefined'
hackaUtils.replace_empty_lists(filtered_dt, column, nan_class)


Field values (%):  education_level
Bachillerato completo                  0.304918
Técnico                                0.217845
Profesional                            0.210709
Tecnólogo                              0.125889
Especialización/ Maestría              0.065747
Bachillerato (grados 9°, 10° y 11°)    0.031058
NaN                                    0.029767
Básica primaria                        0.007367
Bachillerato (grados 6°, 7° u 8°)      0.006374
Doctorado                              0.000326
Name: education_level, dtype: float64
Field values (value):  education_level
Bachillerato completo                  167206
Técnico                                119458
Profesional                            115545
Tecnólogo                               69033
Especialización/ Maestría               36053
Bachillerato (grados 9°, 10° y 11°)     17031
NaN                                     16323
Básica primaria                          4040
Bachillerato (grados 6°, 7° u 8°)      

### 5. Salary

In [ ]:
# Salary
'''
column = 'salary'
nan_class = 'undefined'
# Cambiar listas vacías por clase 'undefined'
hackaUtils.replace_empty_lists(filtered_dt, column, nan_class)
''''

# Eliminando el campo salary
columns = ['salary']
filtered_dt = hackaUtils.drop_columns(filtered_dt, columns)

__Conclusión:__ Se omite por lo pronto el salario. Tiene una distribución bastante amplia de valores incoherentes, desde valores negativos a valores positivos muy grandes. Es buena idea saber cual es la proporción real de los datos que son útiles, ya que parece haber mucha basura. De todas formas vale la pena hacer un analisis posterior, relacionandolo con el sueldo que se estipula en las vacantes. 

### 6. Without Experience

In [10]:
# Without experience
column = 'without_experience'
props = hackaUtils.count_record_per_class(filtered_dt, column)


Field values (%):  without_experience
False    0.891141
True     0.108859
Name: without_experience, dtype: float64
Field values (value):  without_experience
False    488669
True      59694
Name: without_experience, dtype: int64


### 7. Without Studies

In [11]:
# Without studies
column = 'without_studies'
props = hackaUtils.count_record_per_class(filtered_dt, column)


Field values (%):  without_studies
False    0.916698
True     0.083302
Name: without_studies, dtype: float64
Field values (value):  without_studies
False    502683
True      45680
Name: without_studies, dtype: int64


### 8. Studies

In [ ]:
# Studies
column = 'studies'
nan_class = 'undefined'
# Cambiar listas vacías por clase 'undefined'
hackaUtils.replace_empty_lists(filtered_dt, column, nan_class)


__Conclusión:__ Los datos vienen con json que contienen los tipos de estudios asociados a cada candidato, pueden ser muy variados, aunque algo que si es estandar son las etiquetas de "study_type", que pueden ser "professional", "other", entre otros. Se propone en primera instancia, el criterio de cantidad de estudios, es decir, contar cuantos registros hay en cada uno de esos json. Se podría reemplazar los undefined con 0.

### 9. Experiences

In [ ]:
# Experiences
column = 'experiences'
nan_class = 'undefined'
# Cambiar listas vacías por clase 'undefined'
hackaUtils.replace_empty_lists(filtered_dt, column, nan_class)

__Conclusión:__ Los registros de este campo son similares a los de *studies*, que son json con varios diccionarios o listas por dentro. Se propone de la misma forma el conteo de dichos registros. 